In [0]:
from classes.SMTP import GmailSMTP

In [0]:
# spark.sql("""
# CREATE TABLE IF NOT EXISTS studio.`databricks-notes-001`.notification_metadata (
#     emails STRING
# )
# """)

In [0]:
INVALID_TABLE = "studio.`databricks-notes-001`.invalid"
NOTIFICATION_METADATA_TABLE = "studio.`databricks-notes-001`.notification_metadata"

In [0]:
gmail_address = dbutils.secrets.get(
    scope="gmail-application-secret",
    key="gmail-address"
)
gmail_password = dbutils.secrets.get(
    scope="gmail-application-secret",
    key="gmail-app-password"
)
email_client = GmailSMTP(gmail_address, gmail_password)


In [0]:
stats_df = spark.read.table(INVALID_TABLE).select("filename", "ingestiontimestamp", "ValidationError").groupBy("ingestiontimestamp", "filename", "ValidationError").count()


In [0]:

rows = stats_df.collect()

# Start HTML email body
email_body = """
<html>
  <head>
    <style>
      table {
        border-collapse: collapse;
        width: 100%;
        font-family: Arial, sans-serif;
      }
      th, td {
        border: 1px solid #dddddd;
        text-align: left;
        padding: 8px;
      }
      th {
        background-color: #4CAF50;
        color: white;
      }
      tr:nth-child(even) {
        background-color: #f2f2f2;
      }
      .error {
        color: red;
        font-weight: bold;
      }
    </style>
  </head>
  <body>
    <h2>Data Ingestion Validation Report</h2>
    <p>Below are the validation results for the ingested files:</p>
    <table>
      <tr>
        <th>Ingestion Timestamp</th>
        <th>Filename</th>
        <th>Validation Error</th>
        <th>Count</th>
      </tr>
"""

# Add table rows
for row in rows:
    error_class = "error" if row['ValidationError'] else ""
    email_body += f"""
      <tr>
        <td>{row['ingestiontimestamp']}</td>
        <td>{row['filename']}</td>
        <td class="{error_class}">{row['ValidationError'] or '-'}</td>
        <td>{row['count']}</td>
      </tr>
    """

# Close HTML tags
email_body += """
    </table>
    <p>Generated automatically by the ingestion monitoring system.</p>
  </body>
</html>
"""


In [0]:
notification_metadata_df = spark.read.table(NOTIFICATION_METADATA_TABLE)

for email in notification_metadata_df.select("emails").toPandas()["emails"]:
    email_client.send_email(
        to_address=email,
        subject="Data Ingestion Validation Report",
        body=email_body
    )
